In [1]:
import numpy as np
import time

## Definition 4

In [2]:
def subword_subset(a,n):
    
    split_list = []
    len_a = len(a)
    for i in range(len_a):
        delim = len_a - i
        if delim < n:
            break
        split_list.append(a[i:i+n]) 
    return split_list

def Omega(arr):

#     t0 = time.time()
    
    subword_set = []
    for i in range(1, len(arr)):
        subword_set.extend(subword_subset(arr, i))
    subword_set.append(arr)
    
#     t1 = time.time()
#     print("Total time elapsed: ", t1-t0)
    
    return subword_set

for i in range(10):
    Omega('001010')

In [3]:
Omega1 = Omega('001010')
print(Omega1)
Omega2 = Omega('002020')

['0', '0', '1', '0', '1', '0', '00', '01', '10', '01', '10', '001', '010', '101', '010', '0010', '0101', '1010', '00101', '01010', '001010']


## Previous implementation of Defn 4 
##### Note the difference in efficiency 

In [4]:
def subword_len(a,n=1):
    
    split_list = [a[i:i+n] for i in range(0, len(a), 1)]
    
    return [x  for x in split_list if len(x) == n and len(x)>0]

def subword(a):
    #t0 = time.time()
    try:
        b=[a]
        nmax=len(a)
        for n in range(nmax):
            c=subword_len(a,n)
            if len(c)>0:
                b=np.append(b,c)
        return b

    except: 
        return b

    #t1 = time.time()
    #print("Total time elapsed: ", t1-t0)

subword('34')
# for i in range(10):
#     print(subword('001010'))

array(['34', '3', '4'], dtype='|S2')

## Definition 5 - Jacard Similarity

##### Same as prior implementation as per ishanuc

In [5]:
def get_unweightedJac(S1,S2):
    return (len(set(S1).union(S2))-len(set(S1).intersection(S2)))/(0.0+len(set(S1).union(S2)))

# Definition 6 - Weighted Jacard Sequential Distance

In [6]:
# need to take the sum of the weighting
# Once this is finished, you can test individual parts then move onto putting it all together
# def sum_weight():
    
    


In [7]:
def get_weightedJac(S1,S2, sum_weight):
    sum_weight_union = sum_weight(list(set(S1).union(S2)))
    return (sum_weight_union - sum_weight(list(set(S1).intersection(S2))))/(0.0+sum_weight_union)

# Generate rando data

array([[5, 5, 3],
       [4, 5, 5],
       [4, 0, 5]])

(3,)


In [10]:
# make sure to weight each sequence by population prevalence 
# (basically use the probability vector that we will be shifting around)
# Implement the weighted Jacard Later

def weight(p0_vector, i):
    return 1/p0_vector[i]

def jacMatrix(histories):
    length = histories.shape[0]
    jac_mat = np.zeros(shape = (length, length))
    for i in range(length):
        for j in range(length):
            jac_mat[i][j] = get_unweightedJac(subword(histories[i]), subword(histories[j]))
    return jac_mat
                       
    
    
    


array([[0.        , 0.66666667, 0.75      ],
       [0.66666667, 0.        , 0.33333333],
       [0.75      , 0.33333333, 0.        ]])

# Quasi Metric

In [11]:
# reverse dictionary that assigns probability from original probability vector 
# to an index for use in next fn
def p0_reverse(prob_vec):
    rev_probs = {}
    for i in range(prob_vec.shape[0]):
        rev_probs[i] = prob_vec[i]
    return rev_probs






{0: 0.1, 1: 0.05, 2: 0.05, 3: 0.2, 4: 0.4, 5: 0.2}

In [12]:
# reverse dictionary that assigns probability to indexed history
# we initialize this then we, in the future only edit the hist_prob_dict for future histories
def hist_probs(histories, prob_dict):
    hist_prob = {}
    for i in range(histories.shape[0]):
        hist_prob[i] = prob_dict[histories[i][0]]
    return hist_prob




{0: 0.2, 1: 0.4, 2: 0.4}

In [13]:
# quasi metric matrix normalization constant
def qm_matrix_normalization(hist_probs, i, j):
    return .5 * (((1/hist_probs[i]) + (1/hist_probs[j])) - 2)




1.5

In [14]:
# takes in jacard matrix and probabilty vector
# returns quasi metric matrix
# remember we have not used the weight fn, so this needs to be later implemented
# can combine later with jac_matrix maker so we dont have to do 2 runs of matrix

def qm_Matrix(histories, hist_prob):
    length = len(hist_prob)
    qm_mat = np.zeros(shape = (length, length))
    for i in range(length):
        for j in range(length):
            qm_mat[i][j] = qm_matrix_normalization(hist_prob, i, j) * get_unweightedJac(subword(histories[i]), subword(histories[j]))
    return qm_mat


[[0.         1.83333333 2.0625    ]
 [1.83333333 0.         0.5       ]
 [2.0625     0.5        0.        ]]


# Epsilon Neighborhood Step

In [15]:
# given an epsilon, figures out which indexes are in the epsilon neighborhood of the matrix
# basically for each row, we see which things in epsilon neighborhood then keep indexes,
# which we then use to figure out probabilities of occurrence
# qm matrix is gonna be symmetric, so we do calculations per column
def epsilon_indexes(qm_matrix, epsilon):
    col_indexes = {}
    index_list = []
    for i in range(qm_matrix.shape[0]):
        for j in range(qm_matrix.shape[0]):
            if qm_matrix[i][j] < epsilon:
                index_list.append(j)
        col_indexes[i] = index_list
        index_list = []
    return col_indexes


{0: [0, 1, 2], 1: [0, 1, 2], 2: [0, 1, 2]}

In [27]:
# updates probability of given history
# will cache rpt probabilities later
def hist_prob_update(histories, ep_ind, hist_probs, idx):
    pvecs = hist_probs
    for key in ep_ind:
        yesval = histories[key][idx]
        yes = 0 
        no = 0
        for i in range(len(ep_ind[key])):
            histval = histories[key][ep_ind[key][i]]
            #print(histval)
            if yesval == histval:
                yes = yes + 1
            else:
                no = no + 1
        #print(yes,no)
        prob = yes/(yes * 1.0 + no)
        
        pvecs[key] = pvecs[key] * prob
    return pvecs
        
#hist_prob_update(dummy, test, hist_prob, 1)

In [36]:
# is quasi metric, have to do bigger test : 

p0 = np.asarray([0.1, 0.05, 0.05, 0.2, 0.4, 0.2])
dummy = np.random.choice(np.arange(0, 6), size = (5,5),p = p0)
dummy


array([[2, 5, 5, 4, 3],
       [4, 4, 4, 1, 4],
       [4, 3, 5, 2, 3],
       [4, 2, 0, 0, 4],
       [4, 5, 0, 1, 0]])

# Putting it all together

In [34]:
def House(histories, p0, epsilon):
    p0_rev = p0_reverse(p0)
    
    hist_prob = hist_probs(histories, p0_rev)
    print hist_prob
    #for i in range(len(histories[0])):
    for i in range(2):
        qm = qm_Matrix(histories, hist_prob)
        print(qm)
        epsilon_indices = epsilon_indexes(qm, epsilon)
        print(epsilon_indices)
        hist_prob = hist_prob_update(histories, epsilon_indices, hist_prob, i)
    return hist_prob


In [35]:
print(House(dummy, p0, 3))

{0: 0.2, 1: 0.4, 2: 0.4}
[[0.         1.83333333 2.0625    ]
 [1.83333333 0.         0.5       ]
 [2.0625     0.5        0.        ]]
{0: [0, 1, 2], 1: [0, 1, 2], 2: [0, 1, 2]}
[[0.         4.33333333 4.875     ]
 [4.33333333 0.         2.16666667]
 [4.875      2.16666667 0.        ]]
{0: [0], 1: [1, 2], 2: [1, 2]}
{0: 0.13333333333333333, 1: 0.13333333333333333, 2: 0.06666666666666667}
